In [ ]:
import pandas as pd
import numpy as np
import folium
import seaborn as sns
from folium.plugins import TimestampedGeoJson

import math
import matplotlib.pyplot as plt
from scipy import optimize
#%load_ext line_profiler #%lprun -f help help()
explore_data=False
pd.set_option('display.max_rows', 1000)




In [ ]:
# Load Data
ts_path='csse_covid_19_data/csse_covid_19_time_series/'
df_deaths=pd.read_csv(ts_path+'time_series_19-covid-Deaths.csv')
df_recovered=pd.read_csv(ts_path+'time_series_19-covid-Recovered.csv')
df_confirmed=pd.read_csv(ts_path+'time_series_19-covid-Confirmed.csv')
# Explore data
if(explore_data==True):
    print('Describe Confirmed')
    display(describe(df_confirmed))
    print('Describe Confirmed')
    display(df_confirmed.head())

In [ ]:
# Define function
def describe(df):
    return pd.concat([df.describe().T,df.sum().rename('sum')], axis=1).T
def indloc(df,field,values):
    return(df[df.index.get_level_values(field).isin(values)])
def format_input(df,name): 
    df.rename(columns={'Province/State':'subregion','Country/Region':'region','Lat':'lat','Long':'long'},inplace=True)
    df['subregion'].fillna(df['region']+'_country',inplace=True)
    df = df.set_index(['subregion','region','lat','long']).stack()
    df.index.names = ['subregion','region','lat','long','dates']
    df.name = name
    df = df.reset_index()
    df['dates'] = pd.to_datetime(df['dates'],format='%m/%d/%y')
    return(df)

class Parameter:
    def __init__(self, value):
            self.value = value

    def set(self, value):
            self.value = value

    def __call__(self):
            return self.value

def fit(function, parameters, y, x = None):
    def f(params):
        i = 0
        for p in parameters:
            p.set(params[i])
            i += 1
        return y - function(x)

    if x is None: x = np.arange(y.shape[0])
    p = [param() for param in parameters]
    return optimize.leastsq(f, p)
def latest_data(df):
    temp = df.reset_index('dates')
    return(temp[temp['dates']==temp['dates'].max()])


# Calculate Doubling Rates
def doubling_rate(df,agg_level):
    def base2_exp(x): 
        return a()+(x-c()) * np.log(2) / b()
    lookback = 10000
    sample_size = 10
    double_rate=[]
    df = df[df['days_since_200']>-sample_size]

    for index,group in  df.reset_index().groupby(agg_level)[['days','confirmed']]:
        maxdays = group['days'].max()
        for day in range(maxdays,maxdays-lookback,-1): #calculate doubling rate each day starting 15 days ago
            x_data = np.asarray(group['days'].values)
            y_data = np.asarray(np.log(group['confirmed'].values))
            filters = np.argwhere((x_data<=day) & (x_data>day-sample_size)).T[0] #Use last 10 data points for fit

            x_data = x_data[filters]; y_data = y_data[filters]
            a = Parameter(1);b = Parameter(1.333);c = Parameter(0)

            if(len(x_data)<sample_size):
                break
            
            my_fit = fit(base2_exp, [a,b,c], y_data,x_data)
            rate = my_fit[0][1]
            if(rate==1.333):
                rate=np.nan
            else:
                rate=round(rate,2)
            double_rate.append({agg_level:index,'days':day,'Doubling Rate':rate})

    return(pd.DataFrame(double_rate))

def color_producer(rate):
    color_scale = np.array(['#67001f','#b2182b','#d6604d','#f4a582','#fddbc7','#f7f7f7','#d1e5f0','#92c5de','#4393c3','#2166ac','#053061'])
    scale=1
    col=''
    if(np.isnan(rate)):
        col='grey'
    else:
        if(rate<=0):
            col=color_scale[0]
        for i in range(1,10):
            if(i/scale<=rate and rate<(i+1)/scale):
                col=color_scale[i]
        if(rate>=10/scale):
            col=color_scale[10]
    return(col)
def rgb(minimum, maximum, value):
    minimum, maximum = float(minimum), float(maximum)
    ratio = 2 * (value-minimum) / (maximum - minimum)
    b = int(max(0, 255*(1 - ratio)))
    r = int(max(0, 255*(ratio - 1)))
    g = 255 - b - r
    return r, g, b

def repair_colonizers(df):
    df = df.reset_index(drop=True)
    colonizers = df[df['region']==df['subregion']]['region'].unique()
    colonized = df[(df['region'].isin(colonizers)) & (df['region']!=df['subregion'])]['subregion'].unique()
    for i,row in df.iterrows():

        if(row['subregion'] in colonized):
            df.at[i,'region'] = df.at[i,'subregion']        
            df.at[i,'subregion'] = df.at[i,'subregion']+'_country'        
        if((row['region'] in colonizers) & (row['region']==row['subregion'])):
            df.at[i,'subregion'] = df.at[i,'subregion']+'_country'
    return(df)

def create_geojson_features(df):
    print('> Creating GeoJSON features...')
    features = []
    for _, row in df.iterrows():
        
        
        feature = {
            'type': 'Feature',
            'geometry': {
                'type':'Point', 
                'coordinates':[row['long'],row['lat']]
            },
            'properties': {
                'time': row['dates'].__str__(),
                'style': {'color' : color_producer(row['Doubling Rate'])},
                'icon': 'circle',
                'popup':'{} <br> cases: {} <br> death: {} ({}%)  <br> recovery: {} <br> days2double: {}'.format(row['subregion'],
                                                                                                                row['confirmed'],
                                                                                                                row['death'],
                                                                                                                row['death_pc'],
                                                                                                                row['recovered'],
                                                                                                                row['Doubling Rate']),
                'iconstyle':{
                    'fillColor': color_producer(row['Doubling Rate']),
                    'fillOpacity': 0.8,
                    'stroke': 'false',
                    'radius': float(np.log(row['confirmed']+1))
                }
            }
        }
        features.append(feature)
    return features

def make_map(features):
    print('> Making map...')
    my_map = folium.Map(location=[20,0],height='100%', control_scale=True,  zoom_start=1.6)
#    coords_belgium=[50.5039, 4.4699]
#    pollution_map = folium.Map(location=coords_belgium, control_scale=True, zoom_start=8)

    TimestampedGeoJson(
        {'type': 'FeatureCollection',
        'features': features}
        , period='P1D'
        , add_last_point=True
        , auto_play=False
#        , transition_time=10
        , loop=False
        , duration='P1D'
        , max_speed=50
        , loop_button=True
        , date_options='YYYY/MM/DD'
        , time_slider_drag_update=True
    ).add_to(my_map)
    print('> Done.')
    return my_map


In [ ]:
join_cols = ['subregion','region','lat','long','dates']
df_all = format_input(df_deaths,'death')
df_all = pd.merge(df_all,format_input(df_recovered,'recovered'),right_on=join_cols,left_on=join_cols)
df_all = pd.merge(df_all,format_input(df_confirmed,'confirmed'),right_on=join_cols,left_on=join_cols)
df_all = repair_colonizers(df_all)
df_all = df_all.groupby(join_cols).sum()


# Define days passed since beginning of dataset
temp = df_all.reset_index(['region','lat','long','dates'])
pivotalDates = temp.groupby('subregion')['dates'].min()
temp['days'] = temp.merge(pivotalDates,how='left',on=['subregion'],suffixes=('', '_r')).apply(lambda x: (x['dates']-x['dates_r']).days,axis=1).fillna(0)
df_all = temp.set_index(['region','lat','long','dates'],append=True)

# Define data at various subgroups
df_region = df_all.reset_index(['lat','long']).groupby(['region','dates']).agg({'death':sum,'recovered':sum,'confirmed':sum,'lat':np.mean,'long':np.mean,'days':max})
df_region['death_pc'] = round(df_region['death']/df_region['confirmed']*100,2)
df_all['death_pc'] = round(df_all['death'] /df_all['confirmed']*100,2)

# Define days passed since confirmed cases reached 200
temp = df_region.reset_index('dates')
pivotalDates = temp[temp['confirmed']>200].groupby('region')['dates'].min()
temp['days_since_200'] = temp.merge(pivotalDates,how='left',on=['region'],suffixes=('', '_r')).apply(lambda x: (x['dates']-x['dates_r']).days,axis=1)
df_region = temp.set_index('dates',append=True)
temp = df_all.reset_index(['region','lat','long','dates'])
pivotalDates = temp[temp['confirmed']>200].groupby('subregion')['dates'].min()
temp['days_since_200'] = temp.merge(pivotalDates,how='left',on=['subregion'],suffixes=('', '_r')).apply(lambda x: (x['dates']-x['dates_r']).days,axis=1).fillna(0)
df_all = temp.set_index(['region','lat','long','dates'],append=True)


# Calculate Doubling Rates
df_all = df_all.reset_index().merge(doubling_rate(df_all,'subregion'),how='left',on=['subregion','days']).set_index(['subregion','region','lat','long','dates'])
df_region = df_region.reset_index().merge(doubling_rate(df_region,'region'),how='left',on=['region','days']).set_index(['region','dates'])
df_all['days'] = df_all['days'].astype(int)
df_region['days'] = df_region['days'].astype(int)

#Define material data at various subgroups levels
material_countries = list(df_region.groupby(['region']).max().nlargest(6,'confirmed').index)
[material_countries.append(i) for i in ['Canada','Argentina']]

df_all_material = indloc(df_all,'region',material_countries)
df_region_material = indloc(df_region,'region',material_countries)

# Today's data
df_all_today = latest_data(df_all)
df_region_today = latest_data(df_region)



# Reference curves
max_days_since_200=df_region[['days_since_200']].max().values[0]
reference_curve = pd.DataFrame([i for i in range(int(max_days_since_200)+1)],columns=['days_since_200'])
reference_curve['30% per day']=reference_curve['days_since_200'].apply(lambda x: 200*1.30**x)
reference_curve['100% per 4 days ']=reference_curve['days_since_200'].apply(lambda x: 200*math.pow(2,x/4))

In [ ]:
# Doubling Rates
fig,ax = plt.subplots(figsize=(15,5))
temp = df_region_material[df_region_material['days_since_200']>=0]
temp = temp.reset_index('dates').set_index('days_since_200',append=True)
temp['Doubling Rate'].unstack('region').plot(ax=ax)
plt.ylim(0,10)
plt.ylabel('days until cases double')
plt.xlabel('# days since 200$^{\mathrm{th}}$ case')
plt.grid(which='both')
plt.show()

In [ ]:
# Log scale confirmed cases
fig,ax = plt.subplots(figsize=(15,7))
ax.set_yscale('log')
temp = df_region_material[df_region_material['days_since_200']>=0].set_index('days_since_200',append=True)
temp.groupby(['region','days_since_200'])['confirmed'].max().unstack(['region']).plot(ax=ax)
reference_curve.set_index('days_since_200').plot(ax=ax)
plt.grid(True,axis='both',which='both')
plt.ylabel("# Confirmed Cases")
plt.xlabel("# days since 200$^{\mathrm{th}}$ case")
plt.ylim(200,10**5)
plt.show()

In [ ]:
#Map
data=df_all.reset_index()
df = df_region.reset_index()
missing_countries = data[~data['subregion'].str.contains('_country')]['region'].unique()
df = df[df['region'].isin(missing_countries)]
df['subregion'] = df['region']+'_country'
data = data.append(df[data.columns],ignore_index=True).reset_index(drop=True)

m = []
for i in range(0,2):
    if(i==0):
        filter =data['subregion'].str.contains('_country')
        name = 'covid19_countries.html'
    if(i==1):
        filter =~data['subregion'].str.contains('_country')
        name = 'covid19_cities.html'
        
    data_region = data[filter]
    data_region['subregion'] = data_region['subregion'].replace('_country','')
    features_region = create_geojson_features(data_region)
    m_region = make_map(features_region)
    m.append(m_region)
    m_region.save(name)


